In [ ]:
# search embedded results once they are appeneded into one CSV

In [ ]:
import os
import ast
import math
import time
import urllib
import numpy as np
import pandas as pd
from numpy.linalg import norm

import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv('API_KEY')

In [ ]:
def embed(input):
    response = openai.Embedding.create(
    input=input,
    model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']
    return embeddings
    
def dist(arr2, arr1):
    # Given that arr1 is a np array of lists, and arr2 is a np array
    return np.dot(arr1,arr2)/(norm(arr1, axis=1)*norm(arr2))

def dist2(arr1, arr2):
    # Given that arr1 np array, and arr2 is a np array
    return np.dot(arr1,arr2)/(norm(arr1)*norm(arr2))

In [ ]:
df = pd.read_csv('new_concat_embed_sample.csv')
info = pd.read_csv('Google Sheets.csv').set_index('ID')
df = df.dropna(subset=['Embedding'])

df['Embedding'] = df['Embedding'].apply(lambda x: ast.literal_eval(x))

In [ ]:
prompt = 'ct scanner'
prompt_embed = embed(prompt)

distances = []
for i, row in df.iterrows():
    distances.append(dist2(prompt_embed, row['Embedding']))

# sort distances in ascending order and select top 10 closest results
closest_indices = np.argsort(distances)[::-1][:300]
closest_rows = df.iloc[closest_indices].values.tolist()
closest_distances = [distances[i] for i in closest_indices]

# print closest results in a nicely formatted way
print('Query:', prompt)
print('Closest Results:')
j = 0
for i, (r, distance) in enumerate(zip(closest_rows, closest_distances)):
    if len(r[2]) > 85:
        j += 1
        id = r[0]
        name = info.loc[id, 'Manufacturer']
        sn = info.loc[id, 'Scientific Name']
        sp = info.loc[id, 'Specialty']
        url = info.loc[id, 'Website']
        print(f'\n\n\n --- {i + 1} --- ')
        print(f'----- {j} -----')
        print(f'MFR Name: {name}')
        print(f'MFR ID: {id}')
        print('Cosine Distance:', distance)
        print()
        print(f'MFR URL: {url}')
        print()
        print(f'MFR SN: {sn}')
        print()
        print(f'MFR Specialty: {sp}')
        print()
        print('Page:', urllib.parse.unquote(r[1]))
        print('\tSegment:', r[2])